<a href="https://colab.research.google.com/github/MCygert/handball_classifier/blob/main/handball_video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project made for classifing actions in handball


## Loading data

In [87]:
!ls

drive  sample_data


In [155]:
directory_to_data = "drive/MyDrive/handball"
!ls drive/MyDrive/handball

passes	saves  shots


## 1. We need to split data 80/20 into folders


In [161]:
import os
import shutil

# Create folders for test and train data
os.mkdir('test')
os.mkdir('train')

# Creating subfolders per class in both folders
directories = []
for dirname, dirpath, filename in os.walk(directory_to_data):
  directories.append(dirpath)
directories = directories[0]

for directory in directories:
  os.mkdir(f'test/{directory}')
  os.mkdir(f'train/{directory}')

FileExistsError: ignored

In [166]:
files = []
sorted_by_classes = {}
for dirname, dirpath, filename in os.walk(directory_to_data):
  files.append(filename)
files = files[1:]
# Create dictionary which has{class: list_of_videos}
for i, dictionary in enumerate(directories):
  sorted_by_classes[dictionary] = files[i]  


In [167]:
import random
def split_data(data):
  '''
  Returning 2 lists which are splitted in 10/90 manner 
  '''
  train_data = data.copy()
  test_data = []
  for i in range(round(len(data) / 10)):
    test_data.append(train_data.pop(random.randrange(len(train_data))))
  return train_data, test_data

In [168]:
train_data, test_data = split_data(sorted_by_classes['saves'])

# Check if it's splitting correctly
len(train_data), len(test_data), len(sorted_by_classes['saves'])

(49, 5, 54)

In [170]:
# Create splitted data
train_elements = {}
test_elements = {}
for key in sorted_by_classes.items():
  class_of_video = key[0]
  train, test = split_data(key[1])
  train_elements[class_of_video] = train
  test_elements[class_of_video] = test
test_elements['saves'], test_elements['passes'], test_elements['shots']

(['save47.mp4', 'save36.mp4', 'save32.mp4', 'save9.mp4', 'save12.mp4'],
 ['pass4.mp4',
  'pass311.mp4',
  'pass279.mp4',
  'pass8.mp4',
  'pass299.mp4',
  'pass136.mp4',
  'pass209.mp4',
  'pass99.mp4',
  'pass52.mp4',
  'pass310.mp4',
  'pass284.mp4',
  'pass143.mp4',
  '135.mp4',
  'pass339.mp4',
  'pass105.mp4',
  'pass128.mp4',
  'pass212.mp4',
  'pass265.mp4',
  'pass335.mp4',
  'pass223.mp4',
  'pass249.mp4',
  'pass30.mp4',
  'pass71.mp4',
  'pass153.mp4',
  'pass53.mp4',
  'pass73.mp4',
  '164.mp4',
  'pass90.mp4',
  'pass56.mp4',
  'pass272.mp4',
  'pass9.mp4',
  'pass259.mp4',
  'pass228.mp4',
  'pass217.mp4',
  'pass280.mp4'],
 ['shot49.mp4',
  'shot99.mp4',
  'shot135.mp4',
  'shot34.mp4',
  'shot145.mp4',
  'shot139.mp4',
  'shot151.mp4',
  'shot123.mp4',
  'shot7.mp4',
  'shot36.mp4',
  'shot128.mp4',
  'shot101.mp4',
  'shot88.mp4',
  'shot47.mp4',
  'shot40.mp4'])

In [171]:
def move_files_to_correct_directories(data_dictionary, type_of_data):
  for key in data_dictionary.items():
    for file_name in key[1]:
      shutil.copy(directory_to_data + '/' + key[0] + '/' + file_name, type_of_data + '/' + key[0] + '/' + file_name)

In [172]:
move_files_to_correct_directories(train_elements, 'train')
move_files_to_correct_directories(test_elements, 'test')

In [154]:
!rm -rf test

## 2. Extract jpegs of videos

## 3. summarize it with csv
